In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nbimporter
import pandas as pd

import central_system as cs
import analyze_control as ac
import read_load as rl

import exp_study_case
import exp_house_spec
import exp_general_property

Importing Jupyter notebook from exp_study_case.ipynb
Importing Jupyter notebook from exp_house_spec.ipynb
Importing Jupyter notebook from exp_general_property.ipynb


# 居住者の使い方

本モジュールでは負荷計算等の与条件となる居住者の使い方について示します。  

## 0. 検討ケース

試算するケースを取得します。

In [2]:
case = exp_study_case.get_case()
case

{'region': 6,
 'house': {'a_mr': 29.81,
  'a_or': 51.34,
  'a_a': 120.08,
  'r_env': 2.9555555555555557,
  'insulation': 'h11',
  'solar_gain': 'middle'},
 'system': {'default_heat_source_spec': True,
  'v_hs_rtd_h': 1800.0,
  'v_hs_rtd_c': 1800.0,
  'is_duct_insulated': True,
  'vav_system': False,
  'cap_rtd_h': None,
  'cap_rtd_c': None}}

## 1. 外部モジュールの読み込み

### 住宅の仕様

外部モジュール exp_house_spec.get_house_spec を参照のこと。  

$r_{env}$：床面積の合計に対する外皮の部位の面積の合計の比  
$A_A$：床面積の合計, m<sup>2</sup>  
$A_{MR}$：主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$：その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$：非居室の床面積, m<sup>2</sup>  
$A_{HCZ,R,i}$：標準住戸における暖冷房区画 $i$ の床面積, m<sup>2</sup>  
$A_{HCZ,i}$：暖冷房区画 $i$ の床面積, m<sup>2</sup>  
$A_{prt,i}$：暖冷房区画 $i$ から見た非暖冷房空間の間仕切りの面積, m<sup>2</sup>  
$U_{prt}$：間仕切りの熱貫流率, W/m<sup>2</sup>K  
$Q$：熱損失係数, W/m<sup>2</sup>K  
$\mu_H$：暖房期の日射取得係数, ((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\mu_C$：冷房期の日射取得係数, ((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$V_{vent,i}$：暖冷房区画 $i$ の機械換気量, m<sup>3</sup>/h  

In [3]:
r_env, a_a, a_mr, a_or, a_nr, a_hcz, a_prt, u_prt, q, mu_h, mu_c, v_vent = \
    exp_house_spec.get_house_spec(case['region'], case['house'])
r_env, a_a, a_mr, a_or, a_nr, a_hcz, a_prt, u_prt, q, mu_h, mu_c, v_vent

(2.9555555555555557,
 120.08,
 29.81,
 51.34,
 38.92999999999999,
 array([29.81, 16.56, 13.25, 10.76, 10.77,  3.31,  1.66,  3.31, 13.25,
         4.97, 10.77,  1.66]),
 array([37.99636619, 27.7187943 , 22.17838312, 18.01052093, 18.02725933]),
 2.1739130434782608,
 2.7,
 0.101,
 0.062,
 array([60., 20., 40., 20., 20.]))

### 物性値

外部モジュール exp_general_property を参照のこと。

$\rho$：空気の密度, kg/m<sup>3</sup>  
$c$：空気の比熱, J/kgK  
$L_{wtr}$：水の蒸発潜熱, kJ/kg  
暖冷房期間

In [4]:
rho, c, l_wtr, calender = exp_general_property.get_general_property()
rho, c, l_wtr, calender

(1.2,
 1006.0,
 2434.5296000000003,
 array(['休日', '休日', '休日', ..., '休日', '休日', '休日'], dtype=object))

## 2. 暖冷房期間

暖冷房期間は地域に応じて以下のように定まるとする。

| 地域 | 暖房期間 | 冷房期間 |
| --- | --- | --- |
| 1地域 | 9/24～6/7 | 7/10～8/31 |
| 2地域 | 9/26～6/4 | 7/15～8/31 |
| 3地域 | 9/30～5/31 | 7/10～8/31 |
| 4地域 | 10/1～5/30 | 7/10～8/31 |
| 5地域 | 10/10～5/15 | 7/6～8/31 |
| 6地域 | 11/4～4/21 | 5/30～9/23 |
| 7地域 | 11/26～3/27 | 5/15～10/13 |
| 8地域 | なし | 3/25～12/14 |

In [5]:
heating_period, cooling_period = cs.get_heating_and_cooling_schedule(case['region'])
heating_period, cooling_period

(array([ True,  True,  True, ...,  True,  True,  True]),
 array([False, False, False, ..., False, False, False]))

## 3.  在室人数スケジュール

$$ n_{p,d,t} = n_{p,MR,d,t} + n_{p,OR,d,t} + n_{p,NR,d,t} $$  
$$ n_{p,MR,d,t} = n_{s,p,MR,d,t} \frac{ A_{MR} }{ 29.81 } $$  
$$ n_{p,OR,d,t} = n_{s,p,OR,d,t} \frac{ A_{OR} }{ 51.34 } $$  
$$ n_{p,NR,d,t} = n_{s,p,NR,d,t} \frac{ A_{NR} }{ 38.93 } $$  

$n_{p,d,t}$：日付$d$の時刻$t$における在室人数  
$n_{p,MR,d,t}$：日付$d$の時刻$t$における主たる居室の在室人数  
$n_{p,MR,d,t}$：日付$d$の時刻$t$におけるその他の居室の在室人数  
$n_{p,MR,d,t}$：日付$d$の時刻$t$における非居室の在室人数  
$n_{s,p,MR,d,t}$：日付$d$の時刻$t$における標準住戸の主たる居室の在室人数  
$n_{s,p,MR,d,t}$：日付$d$の時刻$t$における標準住戸のその他の居室の在室人数  
$n_{s,p,MR,d,t}$：日付$d$の時刻$t$における標準住戸の非居室の在室人数  
$A_{MR}$：主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$：その他居室の床面積, m<sup>2</sup>  
$A_{NR}$：非居室の床面積, m<sup>2</sup>  

標準住戸における在室人数（人）

平日

| 居室の種類 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 主たる居室 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 2 | 1 | 1 | 0 | 0 | 1 | 1 | 0 | 0 | 1 | 2 | 2 | 3 | 3 | 2 | 1 | 1 |
| その他の居室 | 4 | 4 | 4 | 4 | 4 | 4 | 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 2 | 3 |
| 非居室 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |

休日

| 居室の種類 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 主たる居室 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 3 | 2 | 2 | 2 | 2 | 1 | 0 | 0 | 2 | 3 | 3 | 4 | 2 | 2 | 1 | 0 |
| その他の居室 | 4 | 4 | 4 | 4 | 4 | 4 | 4 | 3 | 1 | 2 | 2 | 2 | 1 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 2 | 2 | 2 | 3 |
| 非居室 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |

In [6]:
n_p, n_p_mr, n_p_or, n_p_nr = cs.get_n_p(a_mr, a_or, a_nr, calender)
n_p, n_p_mr, n_p_or, n_p_nr

(array([4., 4., 4., ..., 4., 3., 3.]),
 array([0., 0., 0., ..., 2., 1., 0.]),
 array([4., 4., 4., ..., 2., 2., 3.]),
 array([0., 0., 0., ..., 0., 0., 0.]))

## 4. 内部発熱スケジュール

$$ q_{gen,d,t} = q_{gen,MR,d,t} + q_{gen,OR,d,t} + q_{gen,NR,d,t} $$
$$ q_{gen,MR,d,t} = q_{s,gen,MR,d,t} \frac{ A_{MR} }{ 29.81 } $$  
$$ q_{gen,OR,d,t} = q_{s,gen,OR,d,t} \frac{ A_{OR} }{ 51.34 } $$  
$$ q_{gen,NR,d,t} = q_{s,gen,NR,d,t} \frac{ A_{NR} }{ 38.93 } $$  

$q_{gen,d,t}$：日付$d$の時刻$t$における内部発熱, W    
$q_{gen,MR,d,t}$：日付$d$の時刻$t$における主たる居室の内部発熱, W    
$q_{gen,MR,d,t}$：日付$d$の時刻$t$におけるその他の居室の内部発熱, W  
$q_{gen,MR,d,t}$：日付$d$の時刻$t$における非居室の内部発熱, W  
$q_{s,gen,MR,d,t}$：日付$d$の時刻$t$における標準住戸の主たる居室の内部発熱, W    
$q_{s,gen,MR,d,t}$：日付$d$の時刻$t$における標準住戸のその他の居室の内部発熱, W  
$q_{s,gen,MR,d,t}$：日付$d$の時刻$t$における標準住戸の非居室の内部発熱, W  
$A_{MR}$：主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$：その他居室の床面積, m<sup>2</sup>  
$A_{NR}$：非居室の床面積, m<sup>2</sup>  

標準住戸における内部発熱, W

平日

| 居室の種類 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 主たる居室 | 66.9 | 66.9 | 66.9 | 66.9 | 66.9 | 66.9 | 123.9 | 383.6 | 323.2 | 307.3 | 134.8 | 66.9 | 286.7 | 271.2 | 66.9 | 66.9 | 236.9 | 288.6 | 407.8 | 383.1 | 423.1 | 339.1 | 312.9 | 278 |  
| その他の居室 | 18 | 18 | 18 | 18 | 18 | 18 | 18 | 18 | 18 | 398.2 | 18 | 18 | 18 | 18 | 18 | 18 | 18 | 18 | 53 | 53 | 115.5 | 103 | 258.3 | 137.3 |
| 非居室 | 41.5 | 41.5 | 41.5 | 41.5 | 41.5 | 41.5 | 126.1 | 249.9 | 158.3 | 191.3 | 117.5 | 41.5 | 42.5 | 89 | 41.5 | 41.5 | 105.8 | 105.8 | 112.1 | 118.5 | 155.7 | 416.1 | 314.8 | 174.9 |

休日

| 居室の種類 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |  
| 主たる居室 | 66.9 | 66.9 | 66.9 | 66.9 | 66.9 | 66.9 | 66.9 | 66.9 | 440.5 | 443.3 | 515.1 | 488.9 | 422.9 | 174.4 | 66.9 | 66.9 | 237.8 | 407.8 | 383.1 | 326.8 | 339.1 | 339.1 | 312.9 | 66.9 |
| その他の居室 | 18 | 18 | 18 | 18 | 18 | 18 | 18 | 18 | 35.5 | 654.3 | 223 | 223 | 53 | 18 | 18 | 18 | 93 | 93 | 55.5 | 18 | 270 | 168.8 | 270 | 18 |
| 非居室 | 41.5 | 41.5 | 41.5 | 41.5 | 41.5 | 41.5 | 41.5 | 281.3 | 311 | 269.5 | 100.4 | 106.7 | 98.5 | 55.8 | 41.5 | 41.5 | 158.4 | 171.3 | 82.7 | 101.4 | 99.5 | 255.1 | 232.1 | 157.8 |

In [7]:
q_gen, q_gen_mr, q_gen_or, q_gen_nr = cs.get_q_gen(a_mr, a_or, a_nr, calender)
q_gen, q_gen_mr, q_gen_or, q_gen_nr

(array([126.4, 126.4, 126.4, ..., 763. , 815. , 242.7]),
 array([ 66.9,  66.9,  66.9, ..., 339.1, 312.9,  66.9]),
 array([ 18. ,  18. ,  18. , ..., 168.8, 270. ,  18. ]),
 array([ 41.5,  41.5,  41.5, ..., 255.1, 232.1, 157.8]))

## 5. 内部発湿スケジュール

$$ w_{gen,d,t} = w_{gen,MR,d,t} + w_{gen,OR,d,t} + w_{gen,NR,d,t} $$
$$ w_{gen,MR,d,t} = w_{s,gen,MR,d,t} \frac{ A_{MR} }{ 29.81 } $$  
$$ w_{gen,OR,d,t} = w_{s,gen,OR,d,t} \frac{ A_{OR} }{ 51.34 } $$  
$$ w_{gen,NR,d,t} = w_{s,gen,NR,d,t} \frac{ A_{NR} }{ 38.93 } $$  

$w_{gen,d,t}$：日付$d$の時刻$t$における内部発湿, g/h    
$w_{gen,MR,d,t}$：日付$d$の時刻$t$における主たる居室の内部発湿, g/h    
$w_{gen,NR,d,t}$：日付$d$の時刻$t$におけるその他の居室の内部発湿, g/h  
$w_{gen,OR,d,t}$：日付$d$の時刻$t$における非居室の内部発湿, g/h  
$w_{s,gen,MR,d,t}$：日付$d$の時刻$t$における標準住戸の主たる居室の内部発湿, g/h    
$w_{s,gen,NR,d,t}$：日付$d$の時刻$t$における標準住戸のその他の居室の内部発湿, g/h  
$w_{s,gen,OR,d,t}$：日付$d$の時刻$t$における標準住戸の非居室の内部発湿, g/h  
$A_{MR}$：主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$：その他居室の床面積, m<sup>2</sup>  
$A_{NR}$：非居室の床面積, m<sup>2</sup>  

標準住戸における内部発湿, g/h

平日

| 居室の種類 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 主たる居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 25.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 50.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |  
| その他の居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |
| 非居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |

休日

| 居室の種類 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 主たる居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 25.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 50.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |  
| その他の居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |
| 非居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |


In [8]:
w_gen, w_gen_mr, w_gen_or, w_gen_nr = cs.get_w_gen(a_mr, a_or, a_nr, calender)
w_gen, w_gen_mr, w_gen_or, w_gen_nr

(array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]))

## 6. 局所換気スケジュール

局所換気量は、住宅規模に依存しない。

$$ V_{local,d,t} = V_{local,MR,d,t} + V_{local,OR,d,t} + V_{local,NR,d,t} $$

$V_{local,d,t}$：日付$d$時刻$t$における局所換気量, m<sup>3</sup>/h  
$V_{local,MR,d,t}$：日付$d$時刻$t$における主たる居室の局所換気量, m<sup>3</sup>/h  
$V_{local,OR,d,t}$：日付$d$時刻$t$におけるその他の居室の局所換気量, m<sup>3</sup>/h  
$V_{local,NR,d,t}$：日付$d$時刻$t$における非居室の局所換気量, m<sup>3</sup>/h  

平日

| 居室の種類 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 主たる居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 75.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 75.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 150.0 | 150.0 | 0.0 | 0.0 | 0.0 | 0.0 |  
| その他の居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |  
| 非居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 6.0 | 2.0 | 0.0 | 0.8 | 0.0 | 0.0 | 0.8 | 0.0 | 0.0 | 0.0 | 0.8 | 0.8 | 0.8 | 0.8 | 0.8 | 52.0 | 25.0 | 102.8.0 |  

休日

| 居室の種類 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |  
| 主たる居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 75.0 | 0.0 | 0.0 | 0.0 | 75.0 | 0.0 | 0.0 | 0.0 | 0.0 | 150.0 | 150.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |  
| その他の居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |  
| 非居室 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 4.0 | 4.0 | 0.0 | 1.2 | 1.2 | 0.0 | 0.0 | 0.0 | 0.0 | 2.0 | 75.8 | 25.0 | 2.0 | 0.8 | 25.0 | 27.0 | 100.8 |  

In [9]:
v_local, v_local_mr, v_local_or, v_local_nr = cs.get_v_local(calender)
v_local, v_local_mr, v_local_or, v_local_nr

(array([  0. ,   0. ,   0. , ...,  25. ,  27. , 100.8]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([  0. ,   0. ,   0. , ...,  25. ,  27. , 100.8]))

## 7. 設定温湿度

$$ \theta_{set,h} = 20.0 $$

$$ \theta_{set,c} = 27.0 $$

$\theta_{set,h}$：暖房時の設定温度, ℃  
$\theta_{set,c}$：冷房時の設定温度, ℃  

In [10]:
theta_set_h, theta_set_c = cs.get_theta_set()

$$ X_{set,c} = 0.013425743 $$

$X_{set,c}$：冷房時の設置絶対湿度, kg/kgDA ( = 27℃ 60% の時の絶対湿度）

In [11]:
x_set_c = cs.get_x_set()

## 8. 外部モジュールへの引き渡し

本モジュールを外部のモジュールから呼び出して必要な値を取得する場合に利用する関数を定義します。

In [12]:
def get_occupant_usage(region, a_mr, a_or, a_nr, calender):

    heating_period, cooling_period = cs.get_heating_and_cooling_schedule(region)
    
    n_p, _, _, _ = cs.get_n_p(a_mr, a_or, a_nr, calender)
    
    q_gen, _, _, _ = cs.get_q_gen(a_mr, a_or, a_nr, calender)
    
    w_gen, _, _, _ = cs.get_w_gen(a_mr, a_or, a_nr, calender)
    
    v_local, v_local_mr, v_local_or, v_local_nr = cs.get_v_local(calender)
    
    theta_set_h, theta_set_c = cs.get_theta_set()
    
    x_set_c = cs.get_x_set()
    
    return heating_period, cooling_period, n_p, q_gen, w_gen, v_local, v_local_mr, v_local_or, v_local_nr, theta_set_h, theta_set_c, x_set_c